In [86]:
#Library
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('once')

In [87]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,# + columns_to_str(columns),
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [88]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.post(url=URL + endpointApi,# + columns_to_str(columns),
                      data=content,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [89]:
def df_normalize(data):
        df = pd.json_normalize(json.loads(data))

        cols = [col for col in df.columns if col.split('.')[-1] != 'type']
        df = df[cols]

        df.columns = df.columns.map(lambda x: x.split(".")[0])
        return df

Création de la table des taux en EUR : 

on pourrait optimiser encore le nombre de requete API avec un appel pour une currency source seulement si on en a besoin et qu'il n'est pas encore dans notre dataframe
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [90]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, rate_value])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,currency src,rate
0,USD,"0,851208716377"
1,EUR,1
2,JPY,"0,008066467694"
3,BEF,"0,024789352477"
4,CYP,"1,708601441376"
...,...,...
23,LUF,"0,024789352477"
24,VAL,"0,000516456899"
25,SML,"0,000516456899"
26,EUr,"0,01"


[TEST] Taux de convertion entre USD et EUR :

In [91]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]
rate = currencyRateToEUR('USD', df_currency_rate)
rate


'0,851208716377'

[TEST] La database de tous les asset :

In [92]:
data = get_data('asset/')
data = json.loads(data)
df = pd.json_normalize(data)
cols = [col for col in df.columns if col.split('.')[-1] != 'type']
df = df[cols]
df.columns = df.columns.map(lambda x: x.split(".")[0])

df_ETF = df
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,MARKET_PLACE_MIC,PAY_ACCOUNT_ON_DEFAULT,LAST_GROSS_VALUE,LAST_QUOTE_DATE,MARKET_PLACE_CURRENCY,CATEGORY_CIC,CREATION_DATE,FIRST_QUOTE_DATE,LAST_QUOTE_SOURCE,EMIR_CLEARING_ID_TYPE,VL_DELAY,SUB_CATEGORY_CIC,MARKET_CAP_CATEGORY,GICS_SUB_INDUSTRY,INDEXED_RATE,GRAPH_PAST_WEEK,GRAPH_LAST_YEAR,PORT_LINE_COUNT,MARKET_PLACE,BLOOMBERG_ID,IS_DEPOSITARY_QUOTES,MARKET_PLACE_CITY,IS_PORTFOLIO,DAILY_PRICE_COUNT,CITY_OR_ACRONYM_OR_NAME,COUNTRY,LAST_QUOTES_UPDATE_DATE,ASSET_DATABASE_ID,ISIN_CODE,MARKET_PLACE_DESC,GRAPH_PERIOD,INDUSTRY,GRAPH_PAST_YEAR,SEDOL_CODE,asset_custom_classification_bloomberg,COUNTRY_CIC,BLOOMBERG_CODE,GLOBAL_TYPE,ASSET_STATUS,SUB_TYPE,CODE_CIC,ASSET_OR_MARKET_PLACE_COUNTRY,LAST_CLOSE_VALUE_IN_CURR,LAST_CLOSE_VALUE,MODIFICATION_DATE,ASSET PRICE MODE,GICS_INDUSTRY_GROUP,PORT_CONTENT_COUNT,IS_FUND_FED,COUNTRY_ISO_3,REST_UNIQUE_NUMBER,FINAL_UNDERLYING,REST_OBJECT_ID,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,GICS_SECTOR,TYPE,MARKET_PLACE_TELEKURS,GRAPH_LAST_6_MONTH,asset_attr_init,MARKET_PLACE_OR_ASSET_COUNTRY,LABEL,MARKET_PLACE_COUNTRY,quote_value_type,IS_EURONEXT,CURRENCY,FLOATING_VALUE,QUOTE_TYPE,IS_COMPOSITE,FINAL_UNDERLYING_TYPE,IS_FUND_SAVE_SENS,GICS_INDUSTRY,SECTOR,IS_ACCESSIBILITY_PUBLIC,LIQUIDITY_ALGO,IS_EUROLAND,MARKET_PLACE_TYPE,IS_PUBLICLY_TRADED,SUP_SECTOR,STOCK_TYPE,asset_custom_classification_Tab,IS_STRUCTURED_PRODUCT,SUB_SECTOR,ELIGIBLE_PEA,ASSET_MARKET_SUB_GROUP,ELIGIBLE_PEA_PME,asset_fund_info_asset_class_focus,asset_fund_info_strategy,asset_fund_info_maximum_front_end_fee,asset_fund_info_bloomberg_ticker,BOND_RISK_COMPUTATION_MODE,asset_fund_info_is_feeded,asset_fund_info_authorized_for_retail_investors,BOND_WAL_COMPUTATION_MODE,BOND_CREDIT_RISK_COMPUTATION_MODE,asset_fund_info_fund_leverage,asset_fund_info_objective_long,asset_fund_info_is_euro_ucits,asset_fund_info_administrator,BOND_WAM_COMPUTATION_MODE,asset_fund_info_isin_code,asset_fund_info_minimum_investment_shares,asset_fund_info_depositary,asset_fund_info_manager_stated_fee,TRANSPARENCY_LIST,asset_fund_info_cut_off,asset_fund_info_maximum_back_end_fee,asset_fund_info_company,asset_fund_info_vl_date,IS_REAL_ESTATE_FUND,asset_fund_info_manager_stated_perf_fee,asset_fund_info_auditor,BOND_YTM_COMPUTATION_MODE,FUND_TYPE,asset_fund_info_fund_type,asset_fund_info_valorisation_frequency,asset_fund_info_creation_date,MIN_BUY_INITIAL_AMOUNT,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_classification,asset_fund_info_decimalisation,asset_fund_info_amf_date,asset_fund_info_minimum_subscription,asset_fund_info_day_cut_off,asset_fund_info_vl_known,asset_fund_info_geographical_focus,asset_fund_info_subscription_committee,MIN_BUY_AMOUNT,asset_fund_info_abide_with_the_four_criterias,asset_fund_info_reuters_code,REUTERS_CODE,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry,asset_fund_info_management_style,asset_fund_info_rate_output,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,QUOTE_PLACE,TELEKURS_QUOTE_PLACE_CODE,IS_SRD,PORT_BASE_NAV,PORT_VL,PORT_CLEAN_MODE,REAL_TIME_PORT_CASH,PORT_STATUS,PORT_CASH,PORT_NET_EX_CASH_EQUIVALENT,PORT_TYPE,NB_COMPTE_ON_PORTFOLIO,PORT_FEED_TYPE,LOCK_MODE,EAI_FRONTPORT_CASH_BY_CRCY,PORT_CASH_AVAILABLE,EAI_BACKPORT_CASH_BY_CRCY,EAI_PORTFOLIO_FEED_NAME,PORT_IS_HISTORICAL,PORT_NET_EX_CASH,PORT_FEED_CODE,PORT_NEED_RECONCILIATION,PORT_CURRENCY_HEDGE,PORT_CASH_EQUIVALENTS,PRICER_MODE,EAI_PORT_AVAILABLE_CASH,VL_DELAY_VALUE,PORT_RATING_MOODYS_SHORT_TERM,PORT_PERCENT_EQUIVALENT,PORT_RATING_SP_SHORT_TERM,PORT_FIRST_TRANSACTION,PORT_RATING_MOODYS,PORT_RATING_SP,PORT_PERCENT_CASH,PORT_RAT

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [93]:
def get_df_asset(asset_date, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&date={}'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_date))
    df_asset = df_normalize(data)
    return df_asset

df_ETF = get_df_asset('2013-06-14')
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,ASSET_DATABASE_ID,LAST_CLOSE_VALUE_IN_CURR,LABEL,TYPE
0,1845,"12,12 EUR",1&1 DRILLISCH AG,STOCK
1,1846,"44,91 EUR",1000MERCIS,STOCK
2,2122,"30,457 EUR",1818-VEGA EUR CO,FUND
3,2123,"1342,23 EUR",29 HAUSSMANN CROISSANCE EUROPE D,FUND
4,2124,NaN,29 HAUSSMANN EQUILIBRE,FUND
...,...,...,...,...
475,1879,"8,53 EUR",YAMANA GOLD INC,STOCK
476,1997,"7,2 EUR",YMAGIS SA,STOCK
477,1991,"2,49 EUR",ZCCM INVESTMEN - B,STOCK
478,2106,"30,8 USD",ZOETIS INC,STOCK


Différence entre LAST_CLOSE_VALUE_IN_CURR et LAST_CLOSE_VALUE ??


Conversion des valeurs monétaires en EUR :

In [94]:
def column_convert_currency(df_ETF, col, df_currency_rate):
    new_col = []
    for elt in df_ETF[col]:
        if elt is np.nan:
            new_col.append(elt)
            continue
            
        value, currency_name = elt.split(' ')
        rate = currencyRateToEUR(currency_name, df_currency_rate)

        new_value = float(value.replace(',', '.')) * float(rate.replace(',', '.'))
        new_col.append(new_value)
    df_ETF[col] = new_col
    return df_ETF

def df_convert_currency(df_ETF, df_currency_rate):
    """cols_name_to_convert = ['LAST_GROSS_VALUE',
                            'LAST_CLOSE_VALUE_IN_CURR',
                            'LAST_CLOSE_VALUE',
                            'MIN_BUY_AMOUNT',
                            'asset_fund_info_minimum_subscription',
                            'MIN_BUY_INITIAL_AMOUNT'
                           ]"""
    cols_name_to_convert = ['LAST_CLOSE_VALUE_IN_CURR']
    for col in cols_name_to_convert:
        df_ETF = column_convert_currency(df_ETF, col, df_currency_rate)
    return df_ETF

df_ETF_convert = df_convert_currency(df_ETF, df_currency_rate)
df_ETF_convert

,ASSET_DATABASE_ID,LAST_CLOSE_VALUE_IN_CURR,LABEL,TYPE
0,1845,12.120000000000,1&1 DRILLISCH AG,STOCK
1,1846,44.910000000000,1000MERCIS,STOCK
2,2122,30.457000000000,1818-VEGA EUR CO,FUND
3,2123,1342.230000000000,29 HAUSSMANN CROISSANCE EUROPE D,FUND
4,2124,NaN,29 HAUSSMANN EQUILIBRE,FUND
...,...,...,...,...
475,1879,8.530000000000,YAMANA GOLD INC,STOCK
476,1997,7.200000000000,YMAGIS SA,STOCK
477,1991,2.490000000000,ZCCM INVESTMEN - B,STOCK
478,2106,26.217228464412,ZOETIS INC,STOCK


Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [96]:
def get_cotation_values(asset_id, start_date, end_date):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = df_normalize(data)
    
    df_cotation.columns = df_cotation.columns.map(lambda x: x.split(".")[0])
    return df_cotation

df_cotation = get_cotation_values('1845', '2013-06-14', '2019-05-31')
df_cotation
    

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2013-06-14,"12,12","12,12",312048,"12,17","12,12",-10,"12,01","12,13",1845,"0,11","12,075","12,12","0,009159034138","12,01"
1,2013-06-17,"12,61","12,61",447491,"12,65","12,61",-10,"12,085","12,655",1845,"0,49","12,575","12,61","0,040429042904","12,085"
2,2013-06-18,"12,88","12,88",300052,"12,88","12,88",-10,"12,555","12,905",1845,"0,27","12,825","12,88","0,021411578113","12,605"
3,2013-06-19,"12,815","12,815",260988,"12,96","12,815",-10,"12,71","12,805",1845,"-0,065","12,795","12,815","-0,005046583851","12,865"
4,2013-06-20,"12,37","12,37",263978,"12,775","12,37",-10,"12,37","12,435",1845,"-0,445","12,355","12,37","-0,034724931721","12,71"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,2019-05-27,"36,006862268298","36,006862268298",NaN,NaN,"36,006862268298",-10,NaN,NaN,1845,"0,539152338935",NaN,"36,006862268298","0,015201216543",NaN
1520,2019-05-28,"36,067253192136","36,067253192136",NaN,NaN,"36,067253192136",-10,NaN,NaN,1845,"0,060390923838",NaN,"36,067253192136","0,001677205956",NaN
1521,2019-05-29,"36,31169724569","36,31169724569",NaN,NaN,"36,31169724569",-10,NaN,NaN,1845,"0,244444053554",NaN,"36,31169724569","0,006777451342",NaN
1522,2019-05-30,"35,796349125318","35,796349125318",NaN,NaN,"35,796349125318",-10,NaN,NaN,1845,"-0,515348120373",NaN,"35,796349125318","-0,014192344601",NaN


Récupération de la liste des ratios :

In [97]:
def get_ratio():
    data = get_data('ratio')
    df_ratio = df_normalize(data)
    return df_ratio

df_ratio = get_ratio()
df_ratio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,id,name,is_benchmark_needed,is_percent
0,11,Correlation,True,False
1,7,Exposition action,False,True
2,15,Exposition action,False,True
3,8,RDT,False,True
4,13,Rendement,False,True
5,9,Rendement_annualise,False,True
6,12,Sharpe,False,False
7,14,VaR historique,False,True
8,10,Volatilite,False,True


[FIX portfolio ID] Récupération de la composition d’un portefeuille :

In [98]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = df_normalize(data)
    return df_portfolio
    #keep seulement list assets ?

portfolio = get_portfolio(564)
df_portfolio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,error_message
0,564 not found


[FIX portfolio ID] MAJ de la composition d’un portefeuille :


In [99]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"], asset["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets):
    label = "EPITA_PTF_1"
    currency = "EUR"
    date = "2013-06-14"
    assets = portfolio_json_assets(assets) 
    res = '[{{"label": "{}", "currency": {{"code": "{}"}}, "type": "front", "values": {{"{}": [{}]}}}}]'.format(label, currency, date, assets)
    return res

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset['asset'])
    return assets

json_portfolio = json.loads(seralize_portfolio_content('', [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]))
print(json_portfolio)

assets = deseralize_portfolio_content(json_portfolio)
assets

[{'label': 'EPITA_PTF_1', 'currency': {'code': 'EUR'}, 'type': 'front', 'values': {'2013-06-14': [{'asset': {'asset': 42, 'quantity': 2}}, {'asset': {'asset': 1, 'quantity': 1}}]}}]


[{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]

In [100]:
def put_portfolio(portfolio_id, assets):
    #data = 
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), assets)
    #df_portfolio = df_normalize(data)
    #return df_portfolio

#df_portfolio = put_portfolio(564, '[]')
#df_portfolio
assets = [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]
content = seralize_portfolio_content('', assets)

put_portfolio(564, content)

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


[TODO] Calcul de ratios :